# 建立回歸模型：準備和視覺化數據

## **南瓜的線性回歸 - 第二課**
#### 簡介

現在你已經準備好使用 Tidymodels 和 Tidyverse 開始構建機器學習模型，你可以開始向數據提出問題了。在處理數據並應用機器學習解決方案時，理解如何提出正確的問題以充分發掘數據集的潛力是非常重要的。

在這一課中，你將學習：

-   如何為模型構建準備數據。

-   如何使用 `ggplot2` 進行數據視覺化。

你需要回答的問題將決定你會使用哪種類型的機器學習算法。而你得到的答案質量將在很大程度上取決於數據的性質。

讓我們通過一個實際練習來看看這一點。

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>插圖由 @allison_horst 提供</figcaption>


<!--![插圖由 \@allison_horst 提供](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.hk.jpg)<br>插圖由 \@allison_horst 提供-->


## 1. 匯入南瓜數據並召喚 Tidyverse

我們需要以下套件來進行本課的數據處理：

-   `tidyverse`：[tidyverse](https://www.tidyverse.org/) 是一個[集合的 R 套件](https://www.tidyverse.org/packages)，旨在讓數據科學更快速、更簡單、更有趣！

你可以使用以下指令安裝：

`install.packages(c("tidyverse"))`

以下的腳本會檢查你是否擁有完成此模組所需的套件，並在缺少某些套件時為你安裝它們。


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

現在，讓我們啟動一些套件並載入為本課提供的[數據](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv)！


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

一個快速的 `glimpse()` 可以立即顯示數據中有空白值，並且混合了字串（`chr`）和數值數據（`dbl`）。`Date` 是字元類型，還有一個奇怪的欄位叫做 `Package`，其中的數據混合了 `sacks`、`bins` 和其他值。事實上，這份數據有點混亂 😤。

其實，直接獲得一份完全準備好用來建立機器學習模型的數據集並不常見。但不用擔心，在這節課中，你將學習如何使用標準的 R 函式庫來準備一份原始數據集 🧑‍🔧。你還會學到各種技術來視覺化數據。📈📊
<br>

> 溫故知新：管道運算符（`%>%`）通過將一個對象向前傳遞到函數或調用表達式中，按邏輯順序執行操作。你可以把管道運算符理解為在程式碼中表示「然後」的意思。


## 2. 檢查遺漏數據

數據科學家最常遇到的問題之一就是不完整或遺漏的數據。R 用特殊的哨兵值 `NA`（Not Available）來表示遺漏或未知的值。

那麼我們如何知道數據框中是否包含遺漏值呢？
<br>
-   一個直接的方法是使用 R 的基礎函數 `anyNA`，它會返回邏輯對象 `TRUE` 或 `FALSE`


In [ ]:
pumpkins %>% 
  anyNA()

太好了，似乎有些數據遺失了！這是一個不錯的起點。

-   另一種方法是使用函數 `is.na()`，它會以邏輯值 `TRUE` 指出哪些單個欄位元素是遺失的。


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

好吧，已完成工作，但像這樣的大型數據框，要逐行逐列檢查會非常低效，幾乎不可能😴。

-   一個更直觀的方法是計算每列的缺失值總和：


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

更好！雖然有些數據缺失，但可能對當前的任務影響不大。讓我們看看進一步的分析會帶來什麼結果。

> 除了豐富的套件和函數，R 還擁有非常完善的文件說明。例如，可以使用 `help(colSums)` 或 `?colSums` 來了解更多有關該函數的資訊。


## 3. Dplyr：資料操作的語法

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>插圖由 @allison_horst 提供</figcaption>


<!--![插圖由 \@allison_horst 提供](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.hk.png)<br/>插圖由 \@allison_horst 提供-->


[`dplyr`](https://dplyr.tidyverse.org/)，是 Tidyverse 中的一個套件，提供了一套一致的數據操作語法，包含一系列動詞，幫助你解決最常見的數據操作挑戰。在本節中，我們將探討一些 dplyr 的動詞！  
<br>


#### dplyr::select()

`select()` 是 `dplyr` 套件中的一個函數，用於幫助你選擇要保留或排除的欄位。

為了讓你的資料框更容易處理，可以使用 `select()` 刪除一些欄位，只保留你需要的欄位。

例如，在這個練習中，我們的分析將涉及 `Package`、`Low Price`、`High Price` 和 `Date` 這些欄位。讓我們選擇這些欄位。


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` 是 `dplyr` 套件中的一個函數，用於創建或修改欄位，同時保留現有的欄位。

`mutate` 的一般結構如下：

`data %>%   mutate(new_column_name = what_it_contains)`

讓我們使用 `Date` 欄位來試試 `mutate`，進行以下操作：

1. 將日期（目前是字元類型）轉換為月份格式（這些是美國日期格式，因此格式為 `MM/DD/YYYY`）。

2. 從日期中提取月份到一個新欄位。

在 R 中，[lubridate](https://lubridate.tidyverse.org/) 套件讓處理日期時間數據變得更簡單。因此，我們可以使用 `dplyr::mutate()`、`lubridate::mdy()` 和 `lubridate::month()` 來完成上述目標。我們可以刪除 Date 欄位，因為在後續操作中不再需要它。


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

哇哦！🤩

接下來，我們來新增一個名為 `Price` 的新欄位，代表南瓜的平均價格。現在，讓我們取 `Low Price` 和 `High Price` 欄位的平均值來填充新的 Price 欄位。


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

耶！💪

「等等！」你可能在快速瀏覽整個數據集後，用 `View(pumpkins)` 說道，「這裡有點奇怪！」🤔

如果你查看 `Package` 欄位，南瓜是以許多不同的方式出售的。有些是以 `1 1/9 蒲式耳` 的單位出售，有些是以 `1/2 蒲式耳` 的單位出售，有些是按南瓜個數出售，有些是按重量（磅）出售，還有些是以不同寬度的大箱子出售。

讓我們來驗證一下：


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

太棒了！👏

南瓜似乎很難穩定地稱重，所以我們可以篩選出在 `Package` 欄位中包含 *bushel* 字串的南瓜，並將這些南瓜放入一個新的資料框 `new_pumpkins` 中。


#### dplyr::filter() 和 stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html)：建立一個數據子集，只包含符合條件的**行**，在此例中，`Package`列中包含 *bushel* 字串的南瓜。

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html)：檢測字串中是否存在某個模式。

[`stringr`](https://github.com/tidyverse/stringr) 套件提供簡單的函數，用於常見的字串操作。


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

你可以看到我們已經篩選到大約415行左右的數據，這些數據包含以蒲式耳計算的南瓜。🤩  


#### dplyr::case_when()

**但等等！還有一件事要做**

你有沒有注意到每行的蒲式耳數量都不同？你需要將價格標準化，顯示每蒲式耳的價格，而不是每 1 1/9 或 1/2 蒲式耳的價格。是時候做一些數學運算來標準化了。

我們將使用 [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) 函數根據一些條件來*變更* Price 列。`case_when` 允許你向量化多個 `if_else()` 語句。


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

現在，我們可以根據每蒲式耳的測量來分析單位價格。不過，這些對南瓜蒲式耳的研究，正好說明了「了解數據的本質」是多麼`重要`！

> ✅ 根據 [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) 的說法，蒲式耳的重量取決於農產品的種類，因為這是一種體積的測量方式。「例如，一蒲式耳的番茄應該重56磅……葉類和綠色蔬菜佔據更多空間但重量較輕，所以一蒲式耳的菠菜只有20磅。」這一切都相當複雜！我們不如別費心去做蒲式耳到磅的轉換，而是直接按蒲式耳定價。不過，這些對南瓜蒲式耳的研究，正好說明了了解數據的本質是多麼重要！

> ✅ 你有沒有注意到，按半蒲式耳出售的南瓜非常昂貴？你能猜出原因嗎？提示：小南瓜比大南瓜貴得多，可能是因為每蒲式耳裡小南瓜的數量多得多，這是由於一個大而中空的派南瓜佔用了更多未使用的空間。


現在，最後為了冒險的樂趣 💁‍♀️，我們將「Month」欄位移到第一個位置，也就是在「Package」欄位之前。

`dplyr::relocate()` 用於更改欄位的位置。


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

做得好！👌 現在你擁有一個乾淨整潔的數據集，可以用來建立你的新回歸模型！  


## 4. 使用 ggplot2 進行數據視覺化

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>資訊圖表由 Dasani Madipalli 製作</figcaption>


<!--![資訊圖表由 Dasani Madipalli 製作](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.hk.png){width="600"}-->

有一句*智慧*的名言是這樣說的：

> 「簡單的圖表比任何其他工具都能為數據分析師帶來更多的信息。」 --- John Tukey

數據科學家的其中一個角色是展示他們所處理數據的質量和特性。為了達到這個目的，他們通常會創建有趣的視覺化圖表，例如散點圖、折線圖和柱狀圖，來展示數據的不同方面。透過這種方式，他們能夠以視覺化的方式展示數據中難以察覺的關係和差距。

視覺化還可以幫助確定最適合數據的機器學習技術。例如，一個看起來沿著直線分佈的散點圖表明該數據非常適合進行線性回歸分析。

R 提供了多種繪製圖表的系統，而 [`ggplot2`](https://ggplot2.tidyverse.org/index.html) 是其中最優雅且最靈活的一種。`ggplot2` 允許你通過**組合獨立的組件**來構建圖表。

讓我們從一個簡單的散點圖開始，展示 Price 和 Month 這兩列數據。

在這個例子中，我們將從 [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html) 開始，提供一個數據集和美學映射（使用 [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)），然後添加一個層（例如 [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)）來繪製散點圖。


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

這是一個有用的圖表嗎 🤷？有什麼讓你感到驚訝的地方嗎？

它並不特別有用，因為它只是顯示你的數據在某個月份中的分佈點。


### **如何令其更有用？**

要讓圖表顯示有用的數據，通常需要以某種方式對數據進行分組。例如，在我們的情況下，找出每個月南瓜的平均價格可以提供更多有關數據底層模式的洞察。這引導我們進一步了解 **dplyr**：

#### `dplyr::group_by() %>% summarize()`

在 R 中，分組聚合可以輕鬆地通過以下方式計算：

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` 將分析單位從整個數據集更改為個別分組，例如按月分組。

-   `dplyr::summarize()` 創建一個新的數據框，其中每個分組變量有一列，以及每個指定的摘要統計有一列。

例如，我們可以使用 `dplyr::group_by() %>% summarize()` 將南瓜根據 **Month** 列進行分組，然後找出每個月的 **平均價格**。


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

簡潔！✨

像月份這類的分類特徵，用柱狀圖來表示會更好 📊。負責繪製柱狀圖的圖層是 `geom_bar()` 和 `geom_col()`。查看 `?geom_bar` 以了解更多資訊。

讓我們來試試吧！


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩這是一個更有用的數據可視化！看起來南瓜的最高價格出現在九月和十月。這是否符合你的預期？為什麼符合或不符合？

恭喜你完成了第二課 👏！你已經為模型構建準備好了數據，然後通過可視化發掘了更多的洞察！



---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應被視為權威來源。對於重要資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
